In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import altair as alt
import pandas as pd

# Create an output widget
output = widgets.Output()

# Welcome message and initial button
def welcome():
    welcome_text = widgets.HTML(
        value="<h2>Bem-vindo!</h2><p>Obrigado por participar desta pesquisa. A seguir, você será solicitado a responder algumas perguntas sobre sua experiência e percepção sobre liderança transformadora. Clique no botão 'Iniciar' para começar.</p>"
    )
    iniciar_button = widgets.Button(description="Iniciar")

    def on_iniciar_clicked(b):
        with output:
            clear_output(wait=True)  # Clear the welcome message and button
            display(idade, experiencia)
            display(widgets.HTML("<br>"))  # Add space between dropdowns and button
            display(next_button)

    iniciar_button.on_click(on_iniciar_clicked)

    with output:
        display(welcome_text, iniciar_button)

    display(output)

# Widgets for age and experience
idade = widgets.Dropdown(
    options=[('até 20 anos', 1), ('20-30 anos', 2), ('30-40 anos', 3), ('40-50 anos', 4), ('50-60 anos', 5), ('mais de 60 anos', 6)],
    description='Faixa Etária:'
)
experiencia = widgets.Dropdown(
    options=[('até 5 anos', 1), ('5-10 anos', 2), ('10-20 anos', 3), ('20-30 anos', 4), ('mais de 30 anos', 5)],
    description='Experiência:'
)

# Questions for the Likert scale
questions = [
    "Eu tenho uma compreensão clara de onde estamos indo",
    "Eu tenho uma noção clara de onde quer que nossa unidade estará em 5 anos",
    "Eu não tenho ideia de onde a organização está indo",
    "Eu digo coisas que deixam os funcionários orgulhosos de pertencer a essa organização",
    "Eu digo coisas positivas sobre o grupo de trabalho",
    "Eu incentivo as pessoas a ver os ambientes em mudança como situações cheias de oportunidades",
    "Eu desafio os membros da minha equipe a pensar sobre velhos problemas de novas maneiras",
    "Eu tenho ideias que forçam os membros da equipe a repensarem algumas coisas que nunca questionei antes",
    "Eu desafio os membros da minha equipe a repensarem algumas das minhas suposições básicas sobre o meu trabalho",
    "Eu considero os sentimentos pessoais dos membros da minha equipe antes de agir",
    "Eu me comporto de uma maneira que considera as necessidades pessoais dos membros da minha equipe",
    "Eu estou atento a que os interesses dos funcionários recebem a devida consideração",
    "Eu elogio os membros da equipe quando fazem um trabalho acima da média",
    "Eu reconheço a melhora na qualidade do trabalho dos membros da minha equipe",
    "Eu elogio pessoalmente os membros da minha equipe quando fazem um trabalho"
]

# Function to create Likert scale questions
def likert_questions(questions):
    sliders = []
    for question in questions:
        question_label = widgets.HTML(value=f"<b>{question}</b>")
        slider = widgets.IntSlider(
            value=4,
            min=1,
            max=7,
            step=1,
            description='',
            style={'description_width': 'initial'},
            layout={'width': '600px'}
        )
        left_label = widgets.Label(value="Discordo totalmente", layout=widgets.Layout(width='150px'))
        right_label = widgets.Label(value="Concordo plenamente", layout=widgets.Layout(width='150px'))
        labels = widgets.HBox([left_label, right_label], layout=widgets.Layout(justify_content='space-between', width='600px'))
        gray_bar = widgets.HTML(value="<hr style='border: none; height: 2px; background-color: lightgray;' />", layout=widgets.Layout(width='600px'))
        vbox = widgets.VBox([question_label, slider, labels, gray_bar], layout=widgets.Layout(margin='10px 0', width='650px'))
        sliders.append(vbox)
    return sliders

questions_widgets = likert_questions(questions)

# Function to update B
def update_B(change):
    pass  # No need to print the value of B here

# Set up the update function for each slider
for vbox in questions_widgets:
    vbox.children[1].observe(update_B, names='value')  # Observe the slider within the VBox

# Output area for B
output_B = widgets.Output()

# Function to handle the button click
def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        idade_value = idade.value
        experiencia_value = experiencia.value
        likert_values = [vbox.children[1].value for vbox in questions_widgets]  # Get the value from each slider

        A = idade_value + experiencia_value
        B = sum(likert_values) / len(likert_values)

        # Create DataFrame for Altair
        data = pd.DataFrame({
            'A': [A],
            'B': [B]
        })

        # Create quadrant labels
        quadrant_labels = pd.DataFrame({
            'A': [7.5, 2.5, 7.5, 2.5],
            'B': [7.5, 7.5, 2.5, 2.5],
            'label': ['Estrategistas Experientes', 'Profissionais em Ascensão', 'Veteranos Eficazes', 'Novos Visionários']
        })

        # Plot with Altair
        base = alt.Chart(data).mark_point(filled=True, size=100).encode(
            x=alt.X('B:Q', scale=alt.Scale(domain=[0, 10]), title='Liderança Transformadora (LTM)'),
            y=alt.Y('A:Q', scale=alt.Scale(domain=[0, 10]), title='Experiência e Idade Combinadas')
        )
        labels = alt.Chart(quadrant_labels).mark_text(
            align='center',
            baseline='middle',
            fontSize=12,
            dy=-10
        ).encode(
            x='B:Q',
            y='A:Q',
            text='label:N'
        )
        hline = alt.Chart(pd.DataFrame({'y': [5]})).mark_rule(strokeDash=[5,5], color='gray').encode(y='y')
        vline = alt.Chart(pd.DataFrame({'x': [5]})).mark_rule(strokeDash=[5,5], color='gray').encode(x='x')

        chart = base + hline + vline + labels

        display(HTML("<br><b>Aqui está uma representação de suas respostas:</b>"))
        display(chart)
        display(HTML("<br>"))  # Add space after the graph
        thank_you_message = widgets.HTML(
            value="<p>Obrigado pela sua utilização dessa ferramenta, a qual esperamos que seja útil para seu crescimento profissional como um Líder Transformacional. Convidamos você a fazer uma nova avaliação no futuro próximo para averiguar seu progresso. Agradecemos também se você puder fazer uma avaliação rápida desse instrumento através desse <a href='#' target='_blank'>link</a>. Até logo!!</p>"
        )
        display(thank_you_message)

# Button widget for submitting the survey
submit_button = widgets.Button(description="Enviar")
submit_button.on_click(on_button_clicked)

# Button to display Likert scale questions
next_button = widgets.Button(
    description="Avaliação de Liderança Transformadora (LTM)",
    layout=widgets.Layout(width='400px', height='50px')
)

def on_next_button_clicked(b):
    with output:
        clear_output(wait=True)
        for vbox in questions_widgets:
            display(vbox)
        display(widgets.HTML("<br>"))  # Add space before the submit button
        display(submit_button, output_B)

next_button.on_click(on_next_button_clicked)

# Display the welcome message and initial button
welcome()
